## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Example team list and credits:
- Alice Anderson: Conceptualization, Data curation, Methodology, Writing - original draft
- Bob Barker:  Analysis, Software, Visualization
- Charlie Chang: Project administration, Software, Writing - review & editing
- Dani Delgado: Analysis, Background research, Visualization, Writing - original draft

## Research Question

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback



## Background and Prior Work

The global coronavirus outbreak in 2020, called COVID-19, has caused a global pandemic, forcing people to isolate and quarantine from each other. During the lockdown period, people spent most of their time at home and turned to digital entertainment and video games as a way to socialize and de-stress. Therefore, the gaming industry in this period witnessed a peak in gamer activity, play time, sales, and stock values.

In this study, we are interested in finding which games were the most popular from each game mode (single player, multiplayer, online co-op). Seeing what game mode people are most interested in, could give the gaming industry a better understanding of their consumer's desires. At first, the group was interested in researching the top 50 games from each game mode, between the pre-COVID period (2018-2019), the COVID era (2020-2021), and the post-COVID era (2021-2023), across multiple popular platforms such as Steam, Epic Games, Xbox, PlayStation, Nintendo, etc. **However**, not all platforms share their players' statistics to the public. So, we narrowed down our platform domain to only include Steam because it has a public database which is called "Steam Charts" and "SteamDB".

There are multiple research papers conducted about video games' activity and price range analysis over the course of time, such as:
- Aliev et al. (2025): These researchers investigated how the pandemic affected the prices and player reviews of mostly Indie games on Steam. By analyzing SteamDB data, they found that player reviews and activity levels are highly correlated. This study confirmed that SteamDB is a reliable tool for our project, but their work mostly focused on Indie games category while treating AAA - a pricing category - as its own game category, but also, we want to look at the top 50 games overall from each game mode.

- Şener et al. (2021): This paper successfully investigated the broader economic impact of COVID-19 on the gaming industry and showed a significant rise in player activity on Steam during 2020. Their findings gave us a "baseline" of when player activity started to increase, but it did not include other factors like playtime, pricing dynamics, game modes, etc. 

- Toledo (2021): Toledo used consumer **surveys** to study how gaming habits changed during lockdown. The survey showed that games became a bridge to "online social life" during quarantine period. This findings is why we emphasize on comparing Online Co-op and Multiplayer modes against Single Player games, as we wanted to see if that online social trend lasted after the pandemic ends. This is the harder data to categorize due to Steam tags are very misleading by mixing Singleplayers and Multiplayers altogether.

Therefore, as curious gamers and data analysts, we decided to conduct extensive research on what are the top 50 popular games from each game mode on Steam pre, during, and post the COVID-19 era.

References (include links):
1. Aliev, A. R., Eyniyev, R., & Aliyev, T. A. (n.d.). Analyzing Price Dynamics, Activity of Players and Reviews of Popular Indie Games on Steam Post-COVID-19 Pandemic using SteamDB. https://www.mecs-press.org/ijitcs/ijitcs-v17-n3/IJITCS-V17-N3-3.pdf
2. Şener, Mehmet & Yalcin, Turkan & Gulseven, Osman. (2021). The Impact of COVID-19 on the Video Game Industry. SSRN Electronic Journal. https://doi.org/10.2139/ssrn.3766147
3. Toledo, M. (2021). Video Game Habits COVID-19. Journal of Marketing Management and Consumer Behavior, 3(4), 66–89. https://doi.org/10.2139/ssrn.3676004

## Hypothesis


Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Data

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
  - Description of the variables most relevant to this project
  - Descriptions of any shortcomings this dataset has with repsect to the project
- Dataset #2 (if you have more than one!)
  - same as above
- etc

Each dataset deserves either a set of bullet points as above or a few sentences if you prefer that method.

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

In [ ]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [ ]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    # { '': '', 'filename':''},
]

# get_data.get_raw(datafiles, destination_directory='data/00-raw/')

get_data.main()

### Dataset #1 

Instructions: 
1. Change the header from Dataset #1 to something more descriptive of the dataset
2. Write a few paragraphs about this dataset. Make sure to cover
   1. Describe the important metrics, what units they are in, and giv some sense of what they mean.  For example "Fasting blood glucose in units of mg glucose per deciliter of blood.  Normal values for healthy individuals range from 70 to 100 mg/dL.  Values 100-125 are prediabetic and values >125mg/dL indicate diabetes. Values <70 indicate hypoglycemia. Fasting idicates the patient hasn't eaten in the last 8 hours.  If blood glucose is >250 or <50 at any time (regardless of the time of last meal) the patient's life may be in immediate danger"
   2. If there are any major concerns with the dataset, describe them. For example "Dataset is composed of people who are serious enough about eating healthy that they voluntarily downloaded an app dedicated to tracking their eating patterns. This sample is likely biased because of that self-selection. These people own smartphones and may be healthier and may have more disposable income than the average person.  Those who voluntarily log conscientiously and for long amounts of time are also likely even more interested in health than those who download the app and only log a bit before getting tired of it"
3. Use the cell below to 
    1. load the dataset 
    2. make the dataset tidy or demonstrate that it was already tidy
    3. demonstrate the size of the dataset
    4. find out how much data is missing, where its missing, and if its missing at random or seems to have any systematic relationships in its missingness
    5. find and flag any outliers or suspicious entries
    6. clean the data or demonstrate that it was already clean.  You may choose how to deal with missingness (dropna of fillna... how='any' or 'all') and you should justify your choice in some way
    7. You will load raw data from `data/00-raw/`, you will (optionally) write intermediate stages of your work to `data/01-interim` and you will write the final fully wrangled version of your data to `data/02-processed`
4. Optionally you can also show some summary statistics for variables that you think are important to the project
5. Feel free to add more cells here if that's helpful for you


In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE


### Steam250 Data Collection

See instructions above for Dataset #1.  Feel free to keep adding as many more datasets as you need.  Put each new dataset in its own section just like these. 

Lastly if you do have multiple datasets, add another section where you demonstrate how you will join, align, cross-reference or whatever to combine data from the different datasets

Please note that you can always keep adding more datasets in the future if these datasets you turn in for the checkpoint aren't sufficient.  The goal here is demonstrate that you can obtain and wrangle data.  You are not tied down to only use what you turn in right now.

In [ ]:
import pandas as pd

years = [2018, 2019, 2020, 2021, 2022, 2023]
steam_games = pd.read_csv('./data/00-raw/games.csv', index_col=False)

In [ ]:
steam_games.head()

In [ ]:
steam_games_subset = steam_games[['AppID', 'Name', 'Genres']]
steam_games_subset.columns = ['appid', 'name', 'genres']

steam_games_subset.head(10)

In [ ]:
curr_year_df = pd.read_csv('./data/00-raw/2018_top250.csv')

curr_merged_df = curr_year_df.merge(
    steam_games_subset,
    on='appid',
    how='left'
)

curr_merged_df = curr_merged_df[['rank', 'appid', 'name_x', 'num_votes', 'rating', 'genres']]
curr_merged_df.columns = ['rank', 'appid', 'name', 'num_votes', 'rating', 'tags']

curr_merged_df.head()


In [ ]:
type(curr_merged_df['tags'].iloc[0])

In [ ]:
# See unique tags
unique_tags = (
    curr_merged_df['tags']
    .explode()
    .str.strip()
    .unique()
)

print(unique_tags)

In [ ]:
# Convert to list
curr_merged_df['tags'] = (
    curr_merged_df['tags']
    .fillna('')
    .str.split(',')
)

# Normalize play modes
tag_map = {
    'Single-player': 'singleplayer',

    'Multi-player': 'multiplayer',
    'MMO': 'multiplayer',
    'PvP': 'multiplayer',
    'Online PvP': 'multiplayer',
    'LAN PvP': 'multiplayer',
    'Shared/Split Screen PvP': 'multiplayer',
    'Cross-Platform Multiplayer': 'multiplayer',

    'Co-op': 'co-op',
    'Online Co-op': 'co-op',
    'LAN Co-op': 'co-op',
    'Shared/Split Screen Co-op': 'co-op',
}


In [ ]:
def clean_tags(tag_list):
    if not isinstance(tag_list, list):
        return []

    cleaned = []

    for tag in tag_list:
        tag = tag.strip()

        if tag in tag_map:
            cleaned.append(tag_map[tag])

    return list(set(cleaned))

curr_merged_df['tags'] = curr_merged_df['tags'].apply(clean_tags)

In [ ]:
curr_merged_df.head(20)

## Ethics

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Team Expectations 

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them